In [ ]:
from covariant_integrator_library_heavyion import *
from bunch_inits import *
from plotting_variables import *

font_size = 16
# Update the rcParams dictionary to set the font size globally
plt.rcParams.update({'font.size': font_size})

### Call the LW integrator. Two energy loss plots will be generated for each run as a sanity check. Final plot is calculated below, with a cell for inputting warpx results.

In [ ]:
c_ms = 299792458

transv_dist = 1e-4

m_particle_rider = 1.007319468  #proton -amu
#m_particle_driver = 207.2 #lead, amu
m_particle_driver = 199.96 # gold
#m_particle = 0.0005485   #electron -amu
#m_particle_driver = 315 #gold hexafloride, amu

stripped_ions_rider = 1.
stripped_ions_driver = 79.
charge_sign_rider= 1.
charge_sign_driver = 1.
Pz_list = np.linspace(9.584300885e5,9.584295e4,30)
min_dE_MeVs_rider = []
min_dE_MeVs_driver = []
fracs_E_rider = []
fracs_E_driver = []
gams = []
gams_drv = []
bets = []
bets_drv = []
        
for i in range(0,len(Pz_list)):
#for i in range(0,1):
        #Pz_list = np.linspace(9.584300885e5,9.584295e4,30)
        Pz_list = np.linspace(1.28e6,9.584295e4,30)
        
        starting_Pz_rider = Pz_list[i]   #3.25E5 corresponds to 1 TeV
        
        starting_Pz_driver = -starting_Pz_rider/m_particle_driver*m_particle_rider  #actually velocity-like, mass factored later
        transv_mom_rider = 1e-6   #1e-2 is roughly 10keV scale
        transv_mom_driver = transv_mom_rider#*207 # lazy adding lead mass
        starting_distance_rider = 1e-3
        starting_distance_driver = 100.

        sim_type = 2    # type 2 used here, it is for bunch--bunch simulations
        pcount_rider = 1    #bunch counts
        pcount_driver = 1

        init_rider, E_MeV_rest_rider = init_bunch(starting_distance_rider,transv_mom_rider,starting_Pz_rider,stripped_ions_rider,m_particle_rider,transv_dist,pcount_rider,charge_sign_rider)

        init_driver, E_MeV_rest_driver = init_bunch(starting_distance_driver,transv_mom_driver,starting_Pz_driver,stripped_ions_driver,m_particle_driver,-transv_dist,pcount_driver,charge_sign_driver)


        #### parameters not used in this test, but required to run integrator
        bunch_dist = 1E5   #bunch--bunch distance at which the driver bunch has maximum charge (gaussian mean)
        cav_spacing = 1E5  #distance to the subsequent driver bunch
        aperture = 1E5 #this parameter reused for bunch-driven gaussian sigma
        z_cutoff = 0
        wall_pos= 1E5


        #coarse initialization
        # static_steps = 1#set init_steps equal to tot_steps to test static integrator only
        # ret_steps = 25
        # step_size = 2e-6

        #interaction zone
        static_steps = 1
        ret_steps = 5500
        #step_size = 1e-7 #+ 2.3e-6 / ( (i+1) / (25) )
        step_size = 1.8e-8 + (i+1)*6.5e-9 
        print(step_size)
        
        retarded_traj,retarded_drv_traj = retarded_integrator3(static_steps,ret_steps,step_size,wall_pos,aperture,sim_type,init_rider,init_driver,bunch_dist,cav_spacing,z_cutoff)#just use retarded_intergrator2 for retarded calcs
        #retarded_traj,retarded_drv_traj = retarded_integrator3(static_steps,ret_steps,step_size,wall_pos,aperture,sim_type,retarded_traj_pre[-1],retarded_drv_traj_pre[-1],bunch_dist,cav_spacing,z_cutoff)#just use retarded_intergrator2 for retarded calcs
        #retarded_traj,retarded_drv_traj = retarded_traj2,retarded_drv_traj2

        delta_e, delta_e_drv, tot_e, tot_e_drv, delta_e_keV, delta_e_MeV, tot_e_GeV, delta_e_keV_drv, delta_e_MeV_drv, tot_e_GeV_drv, zs, xs, zs_drv, xs_drv, bzs, bxs, bys, bdotxs, bdotys, bdotzs, Pzs, Pxs, Pys, Pts, gammas, tees, bzs_drv, bxs_drv, bys_drv, bdotxs_drv, bdotys_drv, bdotzs_drv, Pzs_drv, Pxs_drv, Pys_drv, Pts_drv, gammas_drv, tees_drv, qs_drv, gam_fixed, bdotzs_seconds, q_statC_squared, q_kg_m3_sneg2, Pows_z_rider_watts = calculate_plotting_variables(retarded_traj,
        retarded_drv_traj, init_rider, init_driver, static_steps, ret_steps, E_MeV_rest_rider, E_MeV_rest_driver, c_ms)
        min_dE_MeVs_rider = np.append(min_dE_MeVs_rider,np.min(delta_e_MeV))
        min_dE_MeVs_driver = np.append(min_dE_MeVs_driver,np.min(delta_e_MeV_drv))
        # max_dE_MeVs_rider = np.append(max_dE_MeVs_rider,np.max(np.abs(delta_e_MeV)))
        # max_dE_MeVs_driver = np.append(max_dE_MeVs_driver,np.max(np.abs(delta_e_MeV_drv)))
        print(zs[-1])
        frac_E_rider = np.divide(np.min(tot_e),tot_e[0])
        frac_E_driver = np.divide(np.min(tot_e_drv),tot_e_drv[0])
        fracs_E_rider = np.append(fracs_E_rider,frac_E_rider)
        fracs_E_driver = np.append(fracs_E_driver,frac_E_driver)
        print("driver fractional energy shift", fracs_E_driver)
        print("rider fractional energy shift", fracs_E_rider)
        gams = np.append(gams,gammas[0])
        gams_drv = np.append(gams_drv,gammas_drv[0])
        bets = np.append(bets,bzs[0])
        bets_drv = np.append(bets_drv,bzs_drv[0])

        fig = plt.figure(figsize=(6,5))
        ax = fig.add_subplot(1, 1, 1) 
        ax.scatter(zs,delta_e_MeV/1000,s=40)
        #ax.scatter(zs_drv,delta_e_MeV_drv,s=40)
        #ax.scatter(tees,np.abs(np.multiply(qs_drv,1E-2)),s=40)
        ax.set_ylabel(r"$\Delta$ E [GeV]")
        ax.set_xlabel(r"z [mm]")
        #ax.axvline(30,color='g',label="wall paosition")
        #plt.xlim(starting_distance_rider/2,3)
        #plt.ylim(1E-2,2e3)
        #plt.xscale('log')  #log scale works at 1mm range
        #plt.yscale('log')
        plt.title("Rider")
        plt.tight_layout()
        #plt.savefig("./6.26_prot_antiprot_10nm.png",dpi=600)
        plt.show()

        fig = plt.figure(figsize=(6,5))
        ax = fig.add_subplot(1, 1, 1) 
        #ax.scatter(zs,delta_e_MeV,s=40)
        ax.scatter(zs_drv,delta_e_MeV_drv/1000,s=40)
        #ax.scatter(zs_drv,np.abs(delta_e_MeV_drv/1000),s=40)  #for log plotting
        #ax.scatter(tees,np.abs(np.multiply(qs_drv,1E-2)),s=40)
        ax.set_ylabel(r"$\Delta$ E [GeV]")
        #ax.set_xlabel(r"z [mm]")
        #ax.axvline(30,color='g',label="wall paosition")
        #plt.xlim(-0.54,0.01)
        #plt.ylim(1E-5,5e3)
        #plt.xscale('log')
        #plt.yscale('log')
        plt.title("Driver")
        plt.tight_layout()
        #plt.savefig("./6.26_prot_antiprot_10nm.png",dpi=600)
        plt.show()





In [ ]:
amu_kg = 1.66053907E-27
c_ms = 299792458 # Speed of light in m/s
mass_kg = m_particle_driver*amu_kg
vz_mmns = Pz_list/(m_particle_driver*gammas_drv[0]) #NOT mass_kg here
vz_ms = vz_mmns*1e6
Pz_kgms  = vz_ms*mass_kg*gammas_drv[0]
E_J     = Pz_kgms*c_ms
E_MeV_drv = E_J*6.242E12

In [ ]:
amu_kg = 1.66053907E-27
c_ms = 299792458 # Speed of light in m/s
mass_kg = m_particle_rider*amu_kg
vz_mmns = Pz_list/(m_particle_rider*gammas[0]) #NOT mass_kg here
vz_ms = vz_mmns*1e6
Pz_kgms  = vz_ms*mass_kg*gammas[0]
E_J     = Pz_kgms*c_ms
E_MeV = E_J*6.242E12

#### The following are for generating $\gamma \beta$ values to input into warpX

In [ ]:
gams

In [ ]:
gams_drv

In [ ]:
gambets = np.multiply(bets,gams)

In [ ]:
gambets_drv = np.multiply(bets_drv,gams_drv)

In [ ]:
gambets

In [ ]:
-gambets_drv

### The outputs from the warpX gamma_parser.py should then be copied in the following cell

In [ ]:
;pwd

In [ ]:
warpx_gams_start = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_coord2_gammasqd0.25/second_row_values.txt')
warpx_gams_end = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_coord2_gammasqd0.25/last_row_values.txt')

#warpx_gams_start = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_coord2_gammasqd1/second_row_values.txt')
#warpx_gams_end = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_coord2_gammasqd1/last_row_values.txt')

#warpx_gams_start = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_coord2_gammsqd0.5_FIXME/second_row_values.txt')
#warpx_gams_end = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_coord2_gammsqd0.5_FIXME/last_row_values.txt')

#warpx_gams_start = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_8_18_noenergyset/second_row_values.txt')
#warpx_gams_end = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_8_18_noenergyset//last_row_values.txt')

#warpx_gams_start = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_coord2_gammasqd4/second_row_values.txt')
#warpx_gams_end = np.loadtxt('/home/benjamif/work/warpx/proton_gold/saved_runs_alt_coord2_gammasqd4/last_row_values.txt')

In [ ]:
warpx_gams_end

In [ ]:
fracs_E_driver_warpx = np.flip(np.divide(warpx_gams_end, warpx_gams_start))
#fracs_E_driver_warpx = (np.divide(warpx_gams_end, warpx_gams_start))

In [ ]:
#fracs_E_driver_warpx_gam_0_5 =  fracs_E_driver_warpx
fracs_E_driver_warpx_gam_0_25 =  fracs_E_driver_warpx
#fracs_E_driver_warpx_gam_1 =  fracs_E_driver_warpx
#fracs_E_driver_warpx_gam_4 =  fracs_E_driver_warpx


In [ ]:
fracs_E_driver

In [ ]:
fracs_E_rider

In [ ]:
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(1, 1, 1) 
#ax.scatter(zs,delta_e_MeV,s=40)
ax.scatter(E_MeV_drv/1e6,np.abs(fracs_E_driver-1)*100,s=60,label="Covariant/LW",alpha=0.95)
ax.scatter(E_MeV_drv/1e6,np.abs(fracs_E_driver_warpx_gam_0_5-1)*100,s=60,marker='h',label="WarpX",alpha=0.95)

ax.fill_between(E_MeV_drv[0::]/1e6,np.abs(fracs_E_driver_warpx_gam_1[0::]-1)*100,np.abs(fracs_E_driver_warpx_gam_0_25[0::]-1)*100, alpha=0.3)
#ax.fill_between(E_MeV_drv[13::]/1e6,np.abs(fracs_E_driver_warpx_gam_4[13::]-1)*100,np.abs(fracs_E_driver_warpx_gam_0_5[13::]-1)*100, alpha=0.5)

#ax.scatter(E_MeV/1e6,np.abs(fracs_E_rider-1)*100,s=40,label="p")
#ax.scatter(zs_drv,np.abs(delta_e_MeV_drv/1000),s=40)  #for log plotting
#ax.scatter(tees,np.abs(np.multiply(qs_drv,1E-2)),s=40)
ax.set_ylabel(r"% Kinetic Energy Loss for Au")
ax.set_xlabel(r"Starting Energy [TeV]")
#ax.axvline(30,color='g',label="wall paosition")
#plt.xlim(-0.54,0.01)
#plt.ylim(1E-5,5e3)
#plt.xscale('log')
plt.legend()
plt.yscale('log')
#plt.title("p$-$Au @ d$_T$=100nm")
plt.tight_layout()
plt.savefig("./p_Au_energy_dependent_v2.png",dpi=600)
plt.show()

In [ ]:
E_MeV_drv[21:-1]/1e6